In [202]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.callbacks.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

import pandas as pd
import numpy as np
import re
import json
import operator

from numpy import array
from numpy import asarray
from numpy import zeros


np.seterr(divide='ignore', invalid='ignore')
import math

import unicodedata
import string
import unidecode


# nltk
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

# spacy
import spacy

import matplotlib.pyplot as plt

In [2]:
df_movies = pd.read_csv('GenresPlotMerged.csv')

In [3]:
df_movies = df_movies.drop(columns=['Unnamed: 0','nan'])

In [4]:
df_movies.columns

Index(['movie_id', 'name', 'release_date', 'revenue', 'runtime', 'Absurdism',
       'Action', 'Addiction', 'Adult', 'Adventure',
       ...
       'War', 'War Film', 'Western', 'Whodunit', 'Women In Prison Films',
       'World cinema', 'Wuxia', 'Z Movie', 'Zombie', 'plot'],
      dtype='object', length=267)

In [5]:
df_genres_onehot = df_movies.drop(columns=['movie_id','name','release_date','revenue','runtime','plot'])
df_genres_onehot.head()

,Absurdism,Action,Addiction,Adult,Adventure,Airplanes And Airports,Albino Bias,Alien,Americana,Animals,...,Vampire,War,War Film,Western,Whodunit,Women In Prison Films,World cinema,Wuxia,Z Movie,Zombie
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_movies.head()

,movie_id,name,release_date,revenue,runtime,Absurdism,Action,Addiction,Adult,Adventure,...,War,War Film,Western,Whodunit,Women In Prison Films,World cinema,Wuxia,Z Movie,Zombie,plot
0,975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,"Set in the second half of the 22nd century, th..."
1,9363483,White Of The Eye,1987,NaN,110.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A series of murders of rich young women throug...
2,261236,A Woman in Flames,1983,NaN,106.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Eva, an upper class housewife, becomes frustra..."
3,18998739,The Sorcerer's Apprentice,2002,NaN,86.0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,"Every hundred years, the evil Morgana returns..."
4,6631279,Little city,1997-04-04,NaN,93.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Adam, a San Francisco-based artist who works a..."


#### Data Pre-Processing

In [8]:
# construct list of stop words
stop_words = set(stopwords.words("english"))
words_to_include = ['not','didn', "didn't", 'doesn', "doesn't",'no']
words_to_remove = ['would','should','could','the','i','u','a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p',
                    'q','r','s','t','u','v','w','x','y','z','ive']

stop_words = [word for word in stop_words if word not in words_to_include]
stop_words = set(stop_words)
stop_words = list(stop_words.union(words_to_remove))

In [32]:
nlp = spacy.load('en')

In [33]:
# function to clean each sentence

def fn_cleanText(review):

    # remove accents from words
    review = unidecode.unidecode(review)
    
    # remove punctuation characters
    alphabets_no_punct = [char for char in review if char not in string.punctuation]
    words_list = "".join(alphabets_no_punct).split() # join each word and then split at spaces

    # remove digits and special characters
    words_list = [re.sub("(\\d|\\W)+"," ",word) for word in words_list]
    
    # convert words to lower case
    words_list = [word.lower() for word in words_list]

    # remove stop words
    review_clean = ' '.join([word for word in words_list if word not in stop_words])
    
    # lemmatize with Spacy
    review_clean = nlp(review_clean)
    review_clean = ' '.join([word.lemma_ for word in review_clean])
    
    # remove extra whitespace
    review_clean = re.sub('  +', ' ', review_clean)

    review_clean = str(review_clean)
    return review_clean

In [34]:
# test block
review = df_movies['plot'][200]
fn_cleanText(review)

'mamarazzi story mother constantly pry live three child often result riotous hilarious situation fifteen year ago violy langit owner small funeral parlor learn suffer medical condition require undergo hysterectomy life -PRON- s want child realize procedure violy no man life mdash no husband no boyfriend no suitor seek help good friend mandy pity love violy mandy offer boyfriend carlo serve sperm donor mandys blessing violy carlo share night fill romance passion mystery mdash night carlo disappear completely live violy mandy violy get wish bless set triplet mdash two girl peachy strawberry one boy dingdong mdash raise single mother child get old violy might ways mean happy kid even extent help son get girl want mimi extremely control overly nosy mother make kid wonder mom intention little child know -PRON- s make family happy add children frustration get mom reveal real identity father tension run high emotion soar story explode flurry shock funny quirky revelation forever change everyo

In [36]:
df_movies['NormalizedPlot'] = df_movies['plot'].apply(fn_cleanText)

In [50]:
df_movies.shape

(42204, 268)

In [2]:
df_movies = pd.read_csv('CleanedPlots.csv')
df_genres_onehot = df_movies.drop(columns=['Unnamed: 0','movie_id','name','release_date','revenue','runtime','plot','NormalizedPlot'])
df_genres_onehot.head()

,Absurdism,Action,Addiction,Adult,Adventure,Airplanes And Airports,Albino Bias,Alien,Americana,Animals,...,Vampire,War,War Film,Western,Whodunit,Women In Prison Films,World cinema,Wuxia,Z Movie,Zombie
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_movies.head()

,Unnamed: 0,movie_id,name,release_date,revenue,runtime,Absurdism,Action,Addiction,Adult,...,War Film,Western,Whodunit,Women In Prison Films,World cinema,Wuxia,Z Movie,Zombie,plot,NormalizedPlot
0,0,975900,Ghosts of Mars,2001-08-24,14010832.0,98.0,0,1,0,0,...,0,1,0,0,0,0,0,0,"Set in the second half of the 22nd century, th...",set second half nd century film depict mars pl...
1,1,9363483,White Of The Eye,1987,NaN,110.0,0,0,0,0,...,0,0,0,0,0,0,0,0,A series of murders of rich young women throug...,series murder rich young woman throughout ariz...
2,2,261236,A Woman in Flames,1983,NaN,106.0,0,0,0,0,...,0,0,0,0,0,0,0,0,"Eva, an upper class housewife, becomes frustra...",eva upper class housewife become frustrated le...
3,3,18998739,The Sorcerer's Apprentice,2002,NaN,86.0,0,0,0,0,...,0,0,0,0,1,0,0,0,"Every hundred years, the evil Morgana returns...",every hundred year evil morgana return claim f...
4,4,6631279,Little city,1997-04-04,NaN,93.0,0,0,0,0,...,0,0,0,0,0,0,0,0,"Adam, a San Francisco-based artist who works a...",adam san franciscobase artist works cab driver...


##### Considering only the top 15 genres

In [25]:
# get all genres
col_names=["movie_id", "freebase_ID","name","release_date","revenue","runtime","languages","countries","genres"]
movie_data = pd.read_csv("movie.metadata.tsv",delimiter="\t",header=None, names=col_names)
movies = movie_data[['movie_id', 'name','release_date','revenue','runtime','genres']]

In [26]:
# Convering Genres from dictionary to a list 

genres = [] 

# extract genres
for i in movies['genres']: 
    genres.append(list(json.loads(i).values()))

In [27]:
# Find genres and frequency
count = 0
g_list = {}
for x in genres :
    for y in x :
        if y not in g_list : 
            g_list[y] = 1
            count +=1
        else :
            g_list[y] = g_list[y] + 1

In [28]:
  # sort the dictionary
temp_sort = sorted(g_list.items(), key=operator.itemgetter(1))

In [31]:
#Create a data frame from sorted list
headers = ['genre','count']
genres_df = pd.DataFrame(temp_sort, columns=headers)
genres_df = genres_df.sort_values(by=['count'], ascending=False)
genres_df.head()

,genre,count
362,Drama,34007
361,Comedy,16349
360,Romance Film,10234
359,Black-and-white,9094
358,Action,8798


In [179]:
# consider only top 15 genres
top_15_genres = genres_df.head(16)
top_15_genres.head()

,genre,count
362,Drama,34007
361,Comedy,16349
360,Romance Film,10234
359,Black-and-white,9094
358,Action,8798


In [156]:
# create list of revelant columns names
genres_list = ['movie_id','name','release_date','revenue','runtime','NormalizedPlot']
for genre in list(top_15_genres['genre']):
    genres_list.append(genre)
genres_list

['movie_id',
 'name',
 'release_date',
 'revenue',
 'runtime',
 'NormalizedPlot',
 'Drama',
 'Comedy',
 'Romance Film',
 'Black-and-white',
 'Action',
 'Thriller',
 'Short Film',
 'World cinema',
 'Crime Fiction',
 'Indie',
 'Documentary',
 'Horror',
 'Silent film',
 'Adventure',
 'Family Film']

In [157]:
genres_list = [genre.lower() for genre in genres_list]

In [158]:
genres_list

['movie_id',
 'name',
 'release_date',
 'revenue',
 'runtime',
 'normalizedplot',
 'drama',
 'comedy',
 'romance film',
 'black-and-white',
 'action',
 'thriller',
 'short film',
 'world cinema',
 'crime fiction',
 'indie',
 'documentary',
 'horror',
 'silent film',
 'adventure',
 'family film']

In [170]:
df_movies = pd.read_csv('CleanedPlots.csv')

In [172]:
# convert all column names to lower case
new_names = []
for colname in df_movies.columns:
    new_names.append(colname.lower())
df_movies.columns = new_names

In [175]:
# subset only required columns from df_movies to get top 12 genres
for column in list(df_movies.columns):
    if column.lower() not in genres_list:
        df_movies.drop(columns=[column], inplace=True)

In [174]:
# test block
for column in genres_list:
    if column not in list(df_movies.columns):
        print(column)

romance film
short film
crime fiction


In [177]:
df_movies.to_csv('TempFile1.csv')

In [182]:
# get the dependant variables in a seperate dataframe
df_genres_onehot = df_movies.drop(columns=['movie_id','name','release_date','revenue','runtime','normalizedplot'])
df_genres_onehot.head()

,action,adventure,black-and-white,comedy,documentary,drama,family film,horror,indie,silent film,thriller,world cinema
0,1,1,0,0,0,0,0,1,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0,0,0,1
4,0,0,0,1,0,1,0,0,0,0,0,0


### Split data into train and test

In [183]:
# set the dependant and independant variables 
X = list(df_movies['normalizedplot']) # list of all plots
y = df_genres_onehot.values # 2D array with one hot encoded genres for each plot

In [184]:
print("Number of plots =",len(X))
print("Number of rows of y =", len(y))

Number of plots = 42204
Number of rows of y = 42204


In [185]:
# Split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [186]:
print("Length of X_train =",len(X_train))
print("Length of X_test=",len(X_test))

Length of X_train = 31653
Length of X_test= 10551


### Get embeddings for each word using Glove

In [187]:
# ----- Convert Words to IDs -----------#

tokenizer = Tokenizer(num_words=5000) 
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train) # generate IDs for each word in each 
X_test = tokenizer.texts_to_sequences(X_test)

In [188]:
# ---- Pad input matrices and standardize it into a 2D array of dimensions: n * 200 where n=len(X_train) ------- #

vocab_size = len(tokenizer.word_index) + 1
maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [192]:
X_train.shape

(31653, 200)

#### Get Glove Vectors for each word in our corpus

In [193]:
embeddings_dictionary = dict() # dictionary store all word-vector pairs from file: glove.6B.100d.txt

glove_file = open('./glove.6B.100d.txt', encoding="utf8")

# parse each line and store the glove file in a dictionary
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [194]:
# each row will corrospond to a word with its 100 dimensional word vector
embedding_matrix = zeros((vocab_size, 100))

# tokenizer.word_index is a list of (word, id) tuples
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

### Build the neural network

In [195]:
# Layer 1 - Input layer of size 200
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(12, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

In [196]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [197]:
print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 200, 100)          14464000  
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dense_3 (Dense)              (None, 12)                1548      
Total params: 14,582,796
Trainable params: 118,796
Non-trainable params: 14,464,000
_________________________________________________________________
None


In [79]:
df_movies.to_csv('CleanedPlots.csv')

##### Fit training data to the model

In [201]:
# create checkpoints to save the model with best val_acc score
filepath=('./AccScores/-{epoch:02d}-{val_loss:.2f}.hdf5')
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

history = model.fit(X_train, y_train, batch_size=128, epochs=5, validation_split=0.2, callbacks=[checkpoint])

Train on 25322 samples, validate on 6331 samples
Epoch 1/5
25322/25322 [==============================] - ETA: 2:02 - loss: 0.2778 - acc: 0.884 - ETA: 2:01 - loss: 0.2624 - acc: 0.892 - ETA: 2:00 - loss: 0.2573 - acc: 0.893 - ETA: 1:59 - loss: 0.2694 - acc: 0.886 - ETA: 1:58 - loss: 0.2707 - acc: 0.885 - ETA: 1:57 - loss: 0.2707 - acc: 0.885 - ETA: 1:58 - loss: 0.2702 - acc: 0.884 - ETA: 1:59 - loss: 0.2702 - acc: 0.885 - ETA: 1:59 - loss: 0.2694 - acc: 0.884 - ETA: 1:59 - loss: 0.2680 - acc: 0.885 - ETA: 1:57 - loss: 0.2658 - acc: 0.886 - ETA: 1:56 - loss: 0.2649 - acc: 0.886 - ETA: 1:56 - loss: 0.2661 - acc: 0.886 - ETA: 1:56 - loss: 0.2655 - acc: 0.887 - ETA: 1:56 - loss: 0.2653 - acc: 0.887 - ETA: 1:55 - loss: 0.2658 - acc: 0.886 - ETA: 1:55 - loss: 0.2664 - acc: 0.886 - ETA: 1:54 - loss: 0.2692 - acc: 0.885 - ETA: 1:54 - loss: 0.2693 - acc: 0.886 - ETA: 1:54 - loss: 0.2691 - acc: 0.886 - ETA: 1:53 - loss: 0.2693 - acc: 0.886 - ETA: 1:52 - loss: 0.2702 - acc: 0.886 - ETA: 1:52 - lo

##### Evaluate model on the test data and report accuracy

In [204]:
# load the saved model with best val_acc
saved_model = load_model('./AccScores/-02-0.28.hdf5')

# evaluate the model
_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
print('Train Accuracy: %.3f, Test Accuracy: %.3f' % (train_acc, test_acc))

Train Accuracy: 0.890, Test Accuracy: 0.881
